In [17]:
import pandas as pd
import numpy as np

# Load the features and targets
features = pd.read_csv('features.csv')
targets = pd.read_csv('targets.csv')

In [18]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix


In [19]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [20]:
# Define the MLP model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(50, activation='sigmoid', input_shape=(X_train_scaled.shape[1],)),  # First hidden layer
    tf.keras.layers.Dense(30, activation='sigmoid'),  # Second hidden layer
    tf.keras.layers.Dense(y_train.shape[1], activation='sigmoid')  # Output layer
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.3),
              loss='categorical_crossentropy')

In [ ]:
# Get predictions
y_pred = model.predict(X_test_scaled)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test.to_numpy(), axis=1)  # Assuming y_test is one-hot encoded


In [ ]:
# Calculate metrics
accuracy = accuracy_score(y_true, y_pred_classes)
error_rate = 1 - accuracy
sensitivity = recall_score(y_true, y_pred_classes, average='macro')
precision = precision_score(y_true, y_pred_classes, average='macro', zero_division=0)
f1 = f1_score(y_true, y_pred_classes, average='macro')


In [ ]:

# Specificity calculation
cm = confusion_matrix(y_true, y_pred_classes)
specificity = np.mean([cm[i][i] / sum(cm[:, i]) for i in range(len(cm))])

# Print metrics
print(f"Accuracy: {accuracy}")
print(f"Error Rate: {error_rate}")
print(f"Sensitivity (Recall): {sensitivity}")
print(f"Specificity: {specificity}")
print(f"Precision: {precision}")
print(f"F1-Score: {f1}")

In [23]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix


# Ensure all features are numeric
features = features.apply(pd.to_numeric, errors='coerce')

# One-hot encode the target variable if it's multi-class classification
targets = pd.get_dummies(targets)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the MLP model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(50, activation='sigmoid', input_shape=(X_train_scaled.shape[1],)),  # First hidden layer
    tf.keras.layers.Dense(30, activation='sigmoid'),  # Second hidden layer
    tf.keras.layers.Dense(y_train.shape[1], activation='softmax')  # Output layer for multi-class classification
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.3),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Custom callback for performance goal
class PerformanceGoalCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs is not None and logs.get('loss') <= 1e-3:  # Performance goal based on loss
            print(f"\nReached performance goal so cancelling training at epoch {epoch}")
            self.model.stop_training = True

# Train the model
history = model.fit(X_train_scaled, y_train, epochs=500, 
                    validation_data=(X_test_scaled, y_test),
                    callbacks=[PerformanceGoalCallback()])

# Get predictions
y_pred = model.predict(X_test_scaled)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test.to_numpy(), axis=1)

# Calculate metrics
accuracy = accuracy_score(y_true, y_pred_classes)
error_rate = 1 - accuracy
sensitivity = recall_score(y_true, y_pred_classes, average='macro')
precision = precision_score(y_true, y_pred_classes, average='macro', zero_division=0)
f1 = f1_score(y_true, y_pred_classes, average='macro')

# Specificity calculation
cm = confusion_matrix(y_true, y_pred_classes)
specificity = np.mean([cm[i][i] / sum(cm[:, i]) for i in range(len(cm))])

# Print metrics
print(f"Accuracy: {accuracy}")
print(f"Error Rate: {error_rate}")
print(f"Sensitivity (Recall): {sensitivity}")
print(f"Specificity: {specificity}")
print(f"Precision: {precision}")
print(f"F1-Score: {f1}")


Epoch 1/500
341/341 [==============================] - 1s 2ms/step - loss: 0.8275 - accuracy: 0.7228 - val_loss: 0.4055 - val_accuracy: 0.8840
Epoch 2/500
341/341 [==============================] - 1s 2ms/step - loss: 0.3355 - accuracy: 0.9032 - val_loss: 0.2779 - val_accuracy: 0.9111
Epoch 3/500
341/341 [==============================] - 1s 2ms/step - loss: 0.2673 - accuracy: 0.9116 - val_loss: 0.3156 - val_accuracy: 0.8891
Epoch 4/500
341/341 [==============================] - 0s 1ms/step - loss: 0.2454 - accuracy: 0.9154 - val_loss: 0.2367 - val_accuracy: 0.9163
Epoch 5/500
341/341 [==============================] - 1s 2ms/step - loss: 0.2344 - accuracy: 0.9158 - val_loss: 0.2195 - val_accuracy: 0.9247
Epoch 6/500
341/341 [==============================] - 1s 2ms/step - loss: 0.2278 - accuracy: 0.9185 - val_loss: 0.2234 - val_accuracy: 0.9243
Epoch 7/500
341/341 [==============================] - 0s 1ms/step - loss: 0.2269 - accuracy: 0.9165 - val_loss: 0.2173 - val_accuracy: 0.9214